In [142]:
import sys
from collections import defaultdict
import itertools

import torch
import torch.nn as nn
from torch.autograd import Variable
import time
from sklearn import metrics
import numpy as np
import json
import torchtext.vocab

import data.gen_one_hot
from scipy import stats

NUM_EPOCHS = 500

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data = "data//9_objs//train.json"
dev_data = "data//9_objs//dev.json"
test_data = "data//9_objs//test.json"
all_data = "data//9_objs//final_data.json"

# We will generate our own one-hot embeddings and throw out the ones that are already in the .json files, if there are any
g = data.gen_one_hot.Gen_data(all_data)
print("output form G",g)

# all the one-hot embedding etc. has to be handled here, rather in data generation,
# because we have to know which word indice go to which real words in order to do the embedding.

#--------------- Model --------------
class Neural_Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Neural_Net, self).__init__()


        #just putting these here so that I can write them in the google sheets
        self.l1 = nn.Linear(input_size, 70, bias = True)
        self.l1_act = nn.ReLU()
        self.l2 = nn.Linear(70,30, bias = True)
        self.l2_act = nn.ReLU()
        self.l3 = nn.Linear(30, output_size, bias = True)
        self.softmax = nn.Softmax()

        

    def forward(self, x):
        out = x
        out = self.l1(out)
        out = self.l1_act(out)
        out = self.l2(out)
        out = self.l2_act(out)
        out = self.l3(out)
        out = self.softmax(out)
        return out

output form G <data.gen_one_hot.Gen_data object at 0x7f38c9e4f828>


In [154]:
#------------------ Loss & Optimizer --------------------
def cross_entropy_loss(p_hats, ps):
    return torch.mean(torch.sum(- ps * torch.log(p_hats), 1))

In [155]:
#-------------- Metrics ---------------------
def cross_entropy(y_true,y_hat):
    return -(y_true * np.log(y_hat)).sum(axis=1).mean()

"""
input: 
    pred: vector containing values that sum to one
    targets: vector containing values that sum to one
outptut:
    boolean indicating if predication was correct
"""
def accuracy(pred, targets):
    top_targs = (np.max(targets) == targets) #set indexes with top values as true
    top_preds = (np.max(pred) == pred) #set indexes with top values as true
    correct_preds = np.sum(top_targs&top_preds) #counts intersections of True
    total_top_preds = np.sum(top_preds)
    accuracy_ratio = correct_preds/total_top_preds
    return accuracy_ratio

In [156]:
#------------ Training The Model -----------------------
def train(model, optimizer, loss_fn, training_data, num_epochs):
    for epoch in range(num_epochs):
        for i, env_data in enumerate(training_data):
            x = Variable(torch.Tensor(env_data["x"]))
            y = Variable(torch.Tensor(env_data["y"]))
            
            y_pred = model(x)
            
            optimizer.zero_grad()
            loss = loss_fn(y_pred, y)
            print(epoch, loss.item(), end='\r')

            loss.backward()
            
            optimizer.step()

In [157]:
#original code for generating data is rather rigid because it needs to work with webppl code. 
#this is a work around for replacing input utterance with synonyms
#so long that the neural model works with an embedding, this will be okay. 
def replace_with_synonyms(utter):
    synonyms = {"laughing":"giggling", "crying":"wailing", "angry":"enraged", 
                "lawyer":"attorney", "doctor":"nurse", "pianist":"musician"}
    return synonyms[utter]
    

In [158]:
# Embedding logic
def read_data_line(line, synonyms = False, embedding=None):
    env_data = eval(line)
    # Construct x out of its parts; throw away any existing x field
    target = env_data["target"]
    env_objs = env_data["env"]
    utter = env_data["utter"] 
    
    if synonyms:
        utter = replace_with_synonyms(utter)
    
    zipped_objs = list(zip(env_objs, target)) # Are there biases in ordering that the network might be picking up on?
    _, env_vec, _ = g.gen_vec(utter, zipped_objs)
    
    env_data['x'] = [env_vec]
    return env_data


In [159]:
def classify(env_data):
    utterance = env_data['utter']
    obj1 = env_data['env'][0]
    obj2 = env_data['env'][1]
    obj3 = env_data['env'][2]
    properties = list(obj1.keys())
    for prop in properties:
        if utterance == obj1[prop] == obj2[prop] == obj3[prop]:
            return "3-ambiguous"
    for prop, otherprop in itertools.permutations(properties):
        for obj1, obj2, obj3 in itertools.permutations([obj1, obj2, obj3]):
            if utterance == obj1[prop] == obj2[prop]:
                if obj1[otherprop] == obj3[otherprop] or obj2[otherprop] == obj3[otherprop]:
                    return "2-pragmatic"
                else:
                    return "2-ambiguous"
    return "unclassified"

In [160]:
def evaluate(model, embedding, data_filename, preds_filename, show_failure_cases=False, synonyms = False):
    count = 0
    aggregated_acc = 0
    aggregated_MSE = 0
    aggregated_CE = 0

    type_totals = defaultdict(list)
    type_wrongs = defaultdict(list)

    with open(preds_filename, 'w') as out: #open file to write results in
        for env_data in open(data_filename, 'r'): #read file with json data
            env_data = read_data_line(env_data, synonyms, embedding) #convert json to matrices
            x = Variable(torch.Tensor(env_data["x"])) #input data
      
            y_hat = model(x).data.numpy()
            y = env_data["y"] #targets
            
            json.dump(env_data, out)
            out.write('\n')

            aggregated_CE += cross_entropy(y,y_hat)

            example_type = classify(env_data)
            type_totals[example_type].append(env_data)
            
            for p_hat,p in zip(y_hat, y):
                accurate = accuracy(p_hat, p)
#                 if not accurate:
#                     type_wrongs[example_type].append(env_data)
#                     if show_failure_cases:
#                         print(env_data['utter'], env_data['env'], env_data['y'], np.exp(y_hat))
                aggregated_acc += accurate
                aggregated_MSE += metrics.mean_squared_error(np.exp(p_hat), p)
                count += 1
            env_data["y_hat"] = np.exp(y_hat).tolist()
            


    avg_acc = aggregated_acc/count
    avg_MSE = aggregated_MSE/count
    avg_CE = aggregated_CE/count

    return (avg_acc, avg_MSE, avg_CE, type_wrongs, type_totals)

In [161]:
def print_eval(train_metrics = (), dev_metrics = (), test_metrics = ()):
    with open("data//9_objs//evaluation.txt",'a') as results:
        for metrics in [train_metrics, dev_metrics, test_metrics]:
            if len(metrics) > 0:
                avg_acc, avg_MSE, avg_CE, type_wrongs, type_totals = metrics[0],metrics[1],metrics[2],metrics[3], metrics[4]
                print("avg acc: ", avg_acc)
                print("avg ce", avg_CE)
                
                results.write("avg acc: %s\n"%str(avg_acc))
                results.write("avg acc: %s\n"%str(avg_CE))
                
                for example_type in type_totals:
                    print("failures on example type %s:" % example_type, len(type_wrongs[example_type]), "/", len(type_totals[example_type]))
                    results.write("failures on example type %s: %i/%i\n" % (example_type, len(type_wrongs[example_type]), len(type_totals[example_type])))
            results.write("#------------------------------------#\n")

In [162]:
def main(embedding=None):
    if embedding == 'glove':
        embedding = torchtext.vocab.GloVe('6B', dim=100)
    elif embedding is not None:
        print("Unknown embedding: %s" % embedding, file=sys.stderr)
        print("Aborting.", file=sys.stderr)
        sys.exit(1)

    with open(all_data) as infile:
        ex_inst = read_data_line(infile.readline(), embedding=embedding)
    print(ex_inst)
    input_size, output_size = len(ex_inst["x"][0]),len(ex_inst["y"][0])

    with open(train_data, 'r') as train_data_file:
        training_data = [read_data_line(line, embedding) for line in train_data_file]

    #for name, param in model.named_parameters():
    #    if param.requires_grad:
    #        print("name: %s, param_data: %s"% (name, param.data))

    loss_fn = cross_entropy_loss 
    model = Neural_Net(input_size, output_size).to(device)
    learning_rate = 1e-3
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = .0001)
    train(model, optimizer, loss_fn, training_data, num_epochs=NUM_EPOCHS)
    
    train_metrics = evaluate(model, embedding, train_data, "train_preds.json", show_failure_cases=False)
    dev_metrics = evaluate(model, embedding, dev_data, "dev_preds.json",show_failure_cases=False)
    test_metrics = evaluate(model, embedding, test_data, "test_preds.json",show_failure_cases=False)
    print_eval(train_metrics, dev_metrics,test_metrics)
    train_synonyms_metrics = evaluate(model, embedding, train_data, "train_preds_w_synonyms.json", synonyms=True, show_failure_cases=False)
    test_synonyms_metrics = evaluate(model, embedding, test_data, "test_preds_w_synonyms.json", synonyms=True, show_failure_cases=False)
    print_eval(train_synonyms_metrics, test_synonyms_metrics)

In [163]:
main()

{'id': 0, 'y': [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]], 'x': [[0.1279296875, 0.00341796875, 0.1953125, 0.283203125, -0.06787109375, -0.087890625, 0.185546875, 0.08984375, 0.205078125, 0.208984375, 0.16015625, -0.314453125, -0.044677734375, 0.31640625, -0.408203125, 0.162109375, 0.0185546875, 0.228515625, 0.09375, -0.2421875, 0.2177734375, 0.1103515625, 0.049560546875, -0.04931640625, 0.043212890625, -0.1787109375, -0.040283203125, 0.00014591217041015625, 0.1484375, -0.10595703125, -0.353515625, 0.07373046875, -0.05517578125, -0.158203125, 0.099609375, -0.287109375, 0.234375, -0.1396484375, 0.138671875, 0.1494140625, 0.248046875, -0.275390625, 0.267578125, -0.318359375, -0.0419921875, 0.23046875, -0.208984375, -0.10888671875, 0.03857421875, 0.255859375, -0.25390625, -0.027587890625, 0.0859375, 0.12890625, 0.15625, 0.0947265625, 0.0986328125, -0.2490234375, 0.25390625, -0.2275390625, -0.2158203125, 0.1328125, -0.1533203125, 0.181640625, 0.1728515625, 0.12158203125,

/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Chan

/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Chan

/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Chan

avg acc:  0.89
avg ce 0.34259342703105233
failures on example type 2-pragmatic: 0 / 32
failures on example type 2-ambiguous: 0 / 6
failures on example type unclassified: 0 / 59
failures on example type 3-ambiguous: 0 / 3
avg acc:  0.7692307692307693
avg ce 0.578493790815656
failures on example type 2-pragmatic: 0 / 1
failures on example type unclassified: 0 / 10
failures on example type 3-ambiguous: 0 / 2
avg acc:  0.8461538461538461
avg ce 1.00236349105029
failures on example type 2-pragmatic: 0 / 3
failures on example type unclassified: 0 / 9
failures on example type 3-ambiguous: 0 / 1


/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Chan

/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Chan

/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Chan

avg acc:  0.84
avg ce 0.6029769544423694
failures on example type 2-pragmatic: 0 / 32
failures on example type 2-ambiguous: 0 / 6
failures on example type unclassified: 0 / 59
failures on example type 3-ambiguous: 0 / 3
avg acc:  0.8461538461538461
avg ce 0.9672465435676993
failures on example type 2-pragmatic: 0 / 3
failures on example type unclassified: 0 / 9
failures on example type 3-ambiguous: 0 / 1


In [47]:
#Seen Cross Entropy
aggregated_CE = 0
count = 0
for env_data in open(test_data, 'r'):
    env_data = read_data_line(env_data)
    y =  np.array(env_data["y"])
    aggregated_CE += stats.entropy(y.T)
    count += 1
print(aggregated_CE/count)

#CE goes to infinity if it goes to zero

[0.23981902]


In [68]:
#seen unseen
def matrice_cross_entropy(y_true,y_hat):
    return -(y_true * np.log(y_hat)).sum(axis=1).mean()

data = []
for env_data in open(test_data, 'r'):
    env_data = read_data_line(env_data)
    y =  np.array(env_data["y"])
    data.append(y[0])
data = np.array(data)

unknown_words_pred = np.full(shape=data.shape, fill_value=.333)
matrice_cross_entropy(data,unknown_words_pred)

1.0996127890016931

In [141]:
def accuracy(pred, targets):
    top_targs = (np.max(targets) == targets) #set indexes with top values as true
    top_preds = (np.max(pred) == pred) #set indexes with top values as true
    correct_preds = np.sum(top_targs&top_preds) #counts intersections of True
    total_top_preds = np.sum(top_preds)
    accuracy_ratio = correct_preds/total_top_preds
    return accuracy_ratio

data = []
for env_data in open(test_data, 'r'):
    env_data = read_data_line(env_data)
    y =  np.array(env_data["y"])

y[0][1] = 1
print(y)
unknown_words_pred = np.full(shape=y.shape, fill_value=.333)
accuracy(unknown_words_pred,y)

[[0 1 1]]


0.6666666666666666